In [1]:
!pip install datasets transformers evaluate scikit-learn

In [2]:
import os
import random
import numpy as np
import pandas as pd
import string
import torch
from torch.utils.data import DataLoader

import evaluate
from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.metrics import f1_score

# ================================================
print("데이터셋 로드 및 설정")
# ================================================

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# 디바이스 설정 (GPU가 사용 가능하면 GPU를 사용하고, 그렇지 않으면 CPU 사용)
#DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 모델과 토크나이저 로드
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 데이터셋 로드
df = pd.read_csv('/data/ephemeral/home/data/train.csv')

dataset = Dataset.from_pandas(df.head(500))

# 데이터셋을 데이터프레임으로 변환
train_ED = pd.DataFrame(df)
df = pd.DataFrame(dataset)

# 데이터셋에 인덱스 컬럼 추가
df = df.reset_index().rename(columns={'index': 'idx'})

데이터셋 로드 및 설정


In [3]:
train_ED

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [ ]:
#구두점을 제거 했을때 예측결과가 잘 나올까에 대한 데이터

In [4]:
# 구두점 문자 정의
punctuation = string.punctuation

# 구두점 개수 계산 함수
def count_punctuation(text):
    return sum(1 for char in text if char in punctuation)

# DataFrame의 텍스트 열에 함수 적용 (예: 'text' 열이 텍스트 데이터를 포함한다고 가정)
df['punctuation_count'] = df['text'].apply(count_punctuation)

# 전체 문자 수 계산
df['total_chars'] = df['text'].str.len()

# 구두점 비율 계산
total_punctuation = df['punctuation_count'].sum()
total_chars = df['total_chars'].sum()

punctuation_ratio = total_punctuation / total_chars

print(f"전체 구두점 비율: {punctuation_ratio:.2%}")

전체 구두점 비율: 7.59%


In [5]:
df_filtered = df[df['punctuation_count'] <= 3]

df_filtered

,idx,ID,text,target,punctuation_count,total_chars
0,0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,3,32
2,2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,3,22
3,3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0,29
4,4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,2,28
5,5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0,30
...,...,...,...,...,...,...
494,494,ynat-v1_train_00494,대형서점엔=없uDtOJ네yC과 손}는 b판사들,0,2,25
495,495,ynat-v1_train_00495,"o시티·리버=L3·4ea확;…아스널 20년만에D""스리그Y좌절",1,3,33
496,496,ynat-v1_train_00496,조소앙이 쓴 독립운I#1평전 유방z *Ep역,0,2,24
497,497,ynat-v1_train_00497,김영란법 때문에E애플 t이폰 초청70못받7 r국 언.,3,1,29


In [ ]:
#불용어 제거 후 자주 나오는 단어 확인

In [6]:
import pandas as pd
from collections import Counter
import re

# 한국어 불용어 리스트 (예시, 필요에 따라 확장 가능)
korean_stopwords = set(['은', '는', '이', '가', '을', '를', '의', '에', '에서', '로', '으로', '과', '와', '도', '만', '에게', '께', '한테', '보다', '라고', '이라고', '으로서', '같이', '처럼', '만큼'])

# 정규표현식 패턴 정의
korean_pattern = re.compile('[가-힣]{2,}')

In [7]:
def process_text(text):
    # 영어와 특수문자 제거, 소문자 변환
    text = re.sub(r'[a-zA-Z0-9\W]+', ' ', text.lower())
    
    # 한글 단어 추출 (2글자 이상)
    words = korean_pattern.findall(text)
    
    # 불용어 제거
    words = [word for word in words if word not in korean_stopwords]
    
    return words

# 모든 텍스트에서 단어 추출 및 빈도 계산
all_words = []
for text in df_filtered['text']:
    all_words.extend(process_text(text))

word_counts = Counter(all_words)

# 상위 50개 단어 추출
top_50_words = word_counts.most_common(50)

# 결과 출력
for word, count in top_50_words:
    print(f"{word}: {count}")

대통령: 8
억원: 8
이란: 7
개발: 6
감독: 6
내년: 6
분기: 5
공개: 5
게시판: 4
목소리: 4
영상: 4
네이버: 4
올해: 4
삼성: 4
김정은: 4
홍콩: 4
세계: 4
협력: 4
우리카드: 3
아이폰: 3
성공: 3
아시안게임: 3
다시: 3
증가: 3
복귀: 3
발견: 3
스마트폰: 3
류현진: 3
모바일: 3
개막: 3
신청: 3
여행: 3
연속: 3
방문: 3
프로농구: 3
교체: 3
구글: 3
출시: 3
남북: 3
만에: 3
민주: 3
신간: 3
애플: 3
개최: 3
정상: 3
환영: 3
그래픽: 3
논란: 3
김정: 2
주말: 2


In [8]:
pip install googletrans==3.1.0a0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
from googletrans import Translator

# 번역기 초기화
translator = Translator()

# 한글 문자 범위를 정의하는 정규표현식
korean_pattern = re.compile('[가-힣]+')

def translate_non_korean(text):
    # 한글이 아닌 문자들을 찾아 리스트로 만듭니다
    non_korean = re.findall(r'[^\s가-힣]+', text)
    
    translated_words = {}
    for word in non_korean:
        try:
            # 한글이 아닌 단어를 한글로 번역
            translated = translator.translate(word, dest='ko').text
            translated_words[word] = translated
            # 원본 텍스트에서 번역된 단어로 교체
            text = text.replace(word, translated)
        except:
            # 번역 실패 시 원래 단어 유지
            pass
    
    return text, translated_words

# df_filtered의 'text' 열에 대해 번역 함수 적용
df_filtered['translated_text'] = df_filtered['text'].apply(lambda x: translate_non_korean(x)[0])
df_filtered['translated_words'] = df_filtered['text'].apply(lambda x: translate_non_korean(x)[1])

# 결과 출력
print(df_filtered[['text', 'translated_text', 'translated_words']])

/tmp/ipykernel_792453/124184933.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['translated_text'] = df_filtered['text'].apply(lambda x: translate_non_korean(x)[0])


                                  text                       translated_text  \
0     정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보   정나 :파1 미사와 함께 KT( 이용기간 에이 단] 큐분종U2보   
2               m 김정) 자주통일 새,?r열1나가야1보            중 김정) 자주통일 새,?아르 자형열1나가야1보   
3        갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩         갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩   
4         pI美대선I앞두고 R2fr단 발] $비해 감시 강화          피메이대선나앞두고 R2fr단 발] $비해 감시 강화   
5       美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다     아름다운성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다   
..                                 ...                                   ...   
494          대형서점엔=없uDtOJ네yC과 손}는 b판사들             대형서점엔=없uDtOJ네yC과 손}는 비판사들   
495  o시티·리버=L3·4ea확;…아스널 20년만에D"스리그Y좌절  그만큼시티·리버=L3·4개확;…아스널 20년만에디"스리그그리고좌절   
496           조소앙이 쓴 독립운I#1평전 유방z *Ep역            조소앙이 쓴 독립운나#1평전 유방와 함께 *엡역   
497      김영란법 때문에E애플 t이폰 초청70못받7 r국 언.   김영란법 때문에그리고애플 티이폰 초청70못받7 아르 자형국 언.   
499                      신:p(피U 로y의 시-                      신:피(피안에 로그리고의 시-   

                                      t

/tmp/ipykernel_792453/124184933.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['translated_words'] = df_filtered['text'].apply(lambda x: translate_non_korean(x)[1])


In [10]:
df_filtered

,idx,ID,text,target,punctuation_count,total_chars,translated_text,translated_words
0,0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,3,32,정나 :파1 미사와 함께 KT( 이용기간 에이 단] 큐분종U2보,"{'i': '나', ':': ':', '1': '1', 'z': '와 함께', 'K..."
2,2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,3,22,"중 김정) 자주통일 새,?아르 자형열1나가야1보","{'m': '중', ')': ')', ',?r': ',?아르 자형', '1': '1'}"
3,3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0,29,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,"{'8': '8', '27': '27', '…': '…'}"
4,4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,2,28,피메이대선나앞두고 R2fr단 발] $비해 감시 강화,"{'pI美': '피메이', 'I': '나', 'R2fr': 'R2fr', ']': ..."
5,5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0,30,아름다운성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,"{'美': '아름다운', '6': '6', '1': '1', '·': '·'}"
...,...,...,...,...,...,...,...,...
494,494,ynat-v1_train_00494,대형서점엔=없uDtOJ네yC과 손}는 b판사들,0,2,25,대형서점엔=없uDtOJ네yC과 손}는 비판사들,"{'=': '=', 'uDtOJ': 'uDtOJ', 'yC': 'yC', '}': ..."
495,495,ynat-v1_train_00495,"o시티·리버=L3·4ea확;…아스널 20년만에D""스리그Y좌절",1,3,33,"그만큼시티·리버=L3·4개확;…아스널 20년만에디""스리그그리고좌절","{'o': '그만큼', '·': '·', '=L3·4ea': '=L3·4개', ';..."
496,496,ynat-v1_train_00496,조소앙이 쓴 독립운I#1평전 유방z *Ep역,0,2,24,조소앙이 쓴 독립운나#1평전 유방와 함께 *엡역,"{'I#1': '나#1', 'z': '와 함께', '*Ep': '*엡'}"
497,497,ynat-v1_train_00497,김영란법 때문에E애플 t이폰 초청70못받7 r국 언.,3,1,29,김영란법 때문에그리고애플 티이폰 초청70못받7 아르 자형국 언.,"{'E': '그리고', 't': '티', '70': '70', '7': '7', '..."


In [18]:
import re
import pandas as pd
from langdetect import detect, LangDetectException

def detect_languages(text):
    # 한글과 공백을 제외한 문자 추출
    non_korean = re.sub(r'[가-힣\s]', '', text)
    
    if not non_korean:
        return set()
    
    languages = set()
    
    # 영어 탐지
    if re.search(r'[a-zA-Z]', non_korean):
        languages.add('English')
    
    # 일본어 탐지 (히라가나, 가타카나)
    if re.search(r'[\u3040-\u30FF]', non_korean):
        languages.add('Japanese')
    
    # 중국어 탐지
    if re.search(r'[\u4E00-\u9FFF]', non_korean):
        languages.add('Chinese')
    
    # 기타 언어 탐지
    try:
        other_lang = detect(non_korean)
        if other_lang not in ['en', 'ja', 'zh-cn', 'zh-tw']:
            languages.add(other_lang)
    except LangDetectException:
        pass
    
    return languages

# 데이터프레임의 각 행에 대해 언어 탐지 함수 적용
df_filtered['detected_languages'] = df_filtered['text'].apply(detect_languages)

# 탐지된 모든 언어 목록 생성
all_detected_languages = set.union(*df_filtered['detected_languages'])

print("탐지된 외국어:")
for lang in all_detected_languages:
    print(f"- {lang}")

# set을 리스트로 변환
df_filtered['detected_languages'] = df_filtered['detected_languages'].apply(list)

# 각 언어별 출현 빈도 계산
language_counts = df_filtered['detected_languages'].explode().value_counts()

print("\n언어별 출현 빈도:")
print(language_counts)

탐지된 외국어:
- no
- vi
- cy
- English
- id
- de
- pl
- ca
- ro
- lt
- da
- fr
- ko
- sk
- es
- tl
- lv
- et
- it
- sv
- sq
- fi
- cs
- so
- sw
- hr
- sl
- af
- pt
- hu
- Chinese

언어별 출현 빈도:
detected_languages
English    185
ko          37
Chinese     36
sw          20
de          17
pl          17
ca          15
so          14
vi          13
sq          11
cy          10
tl           9
no           6
id           5
pt           5
sk           4
cs           4
es           3
sv           3
hu           3
sl           3
it           2
ro           2
fr           2
af           2
da           2
lv           1
lt           1
fi           1
hr           1
et           1
Name: count, dtype: int64


/tmp/ipykernel_792453/3969083926.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['detected_languages'] = df_filtered['text'].apply(detect_languages)
/tmp/ipykernel_792453/3969083926.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['detected_languages'] = df_filtered['detected_languages'].apply(list)


In [20]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

# M2M100 모델 및 토크나이저 로드
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def back_translate(text, src_lang="ko", tgt_lang="en"):
    # 한국어 -> 영어 번역
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    intermediate = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    # 영어 -> 한국어 번역 (Back Translation)
    tokenizer.src_lang = tgt_lang
    encoded = tokenizer(intermediate, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id(src_lang))
    back_translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    return back_translated, intermediate

# Back Translation 수행 및 새 컬럼 추가
df_filtered['Back_Translation_text'] = ""
df_filtered['Back_Translation_words'] = ""

for idx, row in df_filtered.iterrows():
    back_translated, intermediate = back_translate(row['text'])
    df_filtered.at[idx, 'Back_Translation_text'] = back_translated
    df_filtered.at[idx, 'Back_Translation_words'] = {row['text']: back_translated}

print(df_filtered[['text', 'Back_Translation_text', 'Back_Translation_words']])

pytorch_model.bin:  21%|##1       | 409M/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

/tmp/ipykernel_792453/2079577929.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Back_Translation_text'] = ""
/tmp/ipykernel_792453/2079577929.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Back_Translation_words'] = ""


                                  text  \
0     정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보   
2               m 김정) 자주통일 새,?r열1나가야1보   
3        갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩   
4         pI美대선I앞두고 R2fr단 발] $비해 감시 강화   
5       美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다   
..                                 ...   
494          대형서점엔=없uDtOJ네yC과 손}는 b판사들   
495  o시티·리버=L3·4ea확;…아스널 20년만에D"스리그Y좌절   
496           조소앙이 쓴 독립운I#1평전 유방z *Ep역   
497      김영란법 때문에E애플 t이폰 초청70못받7 r국 언.   
499                      신:p(피U 로y의 시-   

                                 Back_Translation_text  \
0                                  P1 MISAZ KT (U2E 만)   
2                                           나는 새인가요? 1   
3                    GNU 8 주말에 열리는 27,000...시장은 불법 보조금   
4       R2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2   
5                           결혼한 여성 6명 중 1명은 결코 빚을 잃었다.   
..                                                 ...   
494                                        판사와 판사가 있다.   
495                  oCity-Rive

In [21]:
df_filtered

,idx,ID,text,target,punctuation_count,total_chars,translated_text,translated_words,foreign_chars,detected_languages,Back_Translation_text,Back_Translation_words
0,0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,3,32,정나 :파1 미사와 함께 KT( 이용기간 에이 단] 큐분종U2보,"{'i': '나', ':': ':', '1': '1', 'z': '와 함께', 'K...",(:KQTU]eiz,"[English, sq]",P1 MISAZ KT (U2E 만),{'정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보': 'P1 MISAZ...
2,2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,3,22,"중 김정) 자주통일 새,?아르 자형열1나가야1보","{'m': '중', ')': ')', ',?r': ',?아르 자형', '1': '1'}",)mr,"[cy, English]",나는 새인가요? 1,"{'m 김정) 자주통일 새,?r열1나가야1보': '나는 새인가요? 1'}"
3,3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0,29,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,"{'8': '8', '27': '27', '…': '…'}",…,[],"GNU 8 주말에 열리는 27,000...시장은 불법 보조금",{'갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩': 'GNU 8 주말에 열...
4,4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,2,28,피메이대선나앞두고 R2fr단 발] $비해 감시 강화,"{'pI美': '피메이', 'I': '나', 'R2fr': 'R2fr', ']': ...",$IR]fpr美,"[no, English, Chinese]",R2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2F2,{'pI美대선I앞두고 R2fr단 발] $비해 감시 강화': 'R2F2F2F2F2F2...
5,5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0,30,아름다운성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,"{'美': '아름다운', '6': '6', '1': '1', '·': '·'}",·美,"[ko, Chinese]",결혼한 여성 6명 중 1명은 결코 빚을 잃었다.,{'美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다': '결혼한 여성 6명 ...
...,...,...,...,...,...,...,...,...,...,...,...,...
494,494,ynat-v1_train_00494,대형서점엔=없uDtOJ네yC과 손}는 b판사들,0,2,25,대형서점엔=없uDtOJ네yC과 손}는 비판사들,"{'=': '=', 'uDtOJ': 'uDtOJ', 'yC': 'yC', '}': ...",=CDJObtuy},"[cy, English]",판사와 판사가 있다.,{'대형서점엔=없uDtOJ네yC과 손}는 b판사들': '판사와 판사가 있다.'}
495,495,ynat-v1_train_00495,"o시티·리버=L3·4ea확;…아스널 20년만에D""스리그Y좌절",1,3,33,"그만큼시티·리버=L3·4개확;…아스널 20년만에디""스리그그리고좌절","{'o': '그만큼', '·': '·', '=L3·4ea': '=L3·4개', ';...",""";=DLYaeo·…","[English, ca]",oCity-River=L3·4ea확; 20년 안에 아르헨티나,"{'o시티·리버=L3·4ea확;…아스널 20년만에D""스리그Y좌절': 'oCity-R..."
496,496,ynat-v1_train_00496,조소앙이 쓴 독립운I#1평전 유방z *Ep역,0,2,24,조소앙이 쓴 독립운나#1평전 유방와 함께 *엡역,"{'I#1': '나#1', 'z': '와 함께', '*Ep': '*엡'}",#*EIpz,"[English, pl]",The Independent I#1 Peace Breastz 에프 스테이션,{'조소앙이 쓴 독립운I#1평전 유방z *Ep역': 'The Independent ...
497,497,ynat-v1_train_00497,김영란법 때문에E애플 t이폰 초청70못받7 r국 언.,3,1,29,김영란법 때문에그리고애플 티이폰 초청70못받7 아르 자형국 언.,"{'E': '그리고', 't': '티', '70': '70', '7': '7', '...",Ert,"[English, da]",Eaple Tifon에 대한 Kim 영국의 법률은 70을 7 개의 방사선을 받지 않...,{'김영란법 때문에E애플 t이폰 초청70못받7 r국 언.': 'Eaple Tifon...


In [ ]:
#노이즈가 포함된 데이터 필터링

In [4]:
train_df = df.reset_index(drop=True)

In [5]:
def introduce_text_noise_df(df, noise_ratio=0.5):
    num_samples = len(df)
    num_noisy = int(noise_ratio * num_samples)
    noisy_indices = random.sample(range(num_samples), num_noisy)
    df_noisy = df.copy()
    for idx in noisy_indices:
        original_text = df_noisy.at[idx, 'text']
        noisy_text = add_noise_to_text(original_text)
        df_noisy.at[idx, 'text'] = noisy_text
    return df_noisy, noisy_indices



def add_noise_to_text(text):
    # 텍스트에 임의의 노이즈를 추가하는 함수 (예: 단어 순서 섞기, 랜덤 단어 추가, 단어 삭제)
    words = text.split()
    random.shuffle(words)

    # 단어 삭제
    num_words_to_delete = random.randint(2, 15)  # 2에서 15개의 단어 삭제
    for _ in range(num_words_to_delete):
        if words:  # 리스트가 비어있지 않은 경우에만 삭제
            delete_position = random.randint(0, len(words) - 1)
            del words[delete_position]

    # 랜덤 단어 추가
    # 랜덤 단어 목록 (한국어)
    random_words = ['사과', '바나나', '자동차', '컴퓨터', '학교', '책', '강아지', '고양이', '하늘', '바다', '산', 'सेब', 'केला', 'कार', 'कंप्यूटर', 'स्कूल', 'किताब', 'कुत्ता', 'बिल्ली', 'आकाश', 'समुद्र', 'पहाड़']
    num_random_words_to_add = random.randint(5, 20)  # 5에서 20개의 랜덤 단어 추가
    for _ in range(num_random_words_to_add):
        random_word = random.choice(random_words)
        insert_position = random.randint(0, len(words))
        words.insert(insert_position, random_word)

    noisy_text = ' '.join(words)
    return noisy_text


# 텍스트 노이즈 적용
train_df_noisy, noisy_text_indices = introduce_text_noise_df(train_df, noise_ratio=0.5)

In [6]:
train_df_noisy

,idx,ID,text,target
0,0,ynat-v1_train_00000,Q분종U2보 고양이 आकाश 고양이 स्कूल कंप्यूटर 고양이 बिल्ली ...,4
1,1,ynat-v1_train_00001,आकाश 산 책 आकाश 자동차 바다 समुद्र बिल्ली किताब सेब स...,3
2,2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,3,ynat-v1_train_00003,किताब कंप्यूटर 얼룩 학교 आकाश 바다 책 पहाड़ 학교 고양이 आक...,5
4,4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
...,...,...,...,...
495,495,ynat-v1_train_00495,"o시티·리버=L3·4ea확;…아스널 20년만에D""스리그Y좌절",1
496,496,ynat-v1_train_00496,강아지 आकाश 책 बिल्ली 컴퓨터 사과 बिल्ली 컴퓨터 केला पहाड़,0
497,497,ynat-v1_train_00497,김영란법 때문에E애플 t이폰 초청70못받7 r국 언.,3
498,498,ynat-v1_train_00498,"स्कूल केला पहाड़ कंप्यूटर 고양이 ""0 바다 आकाश 삶의 학교...",0


In [7]:
# 데이터셋을 다시 Dataset으로 변환
train_dataset_noisy = Dataset.from_pandas(train_df_noisy)

In [8]:
train_dataset_noisy

Dataset({
    features: ['idx', 'ID', 'text', 'target'],
    num_rows: 500
})

In [9]:
# 토크나이저를 사용하여 데이터 전처리
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding=True, truncation=True)
    tokenized_inputs['label'] = examples['target']  # 'target'을 'label'로 매핑
    tokenized_inputs['idx'] = examples['idx']  # 'idx' 유지
    return tokenized_inputs

train_dataset_noisy = train_dataset_noisy.map(
    preprocess_function,
    batched=True,
    remove_columns=['text', 'ID', 'target']  # 'text', 'ID', 'target' 컬럼 제거
)

# 데이터 콜레이터 설정 (동적 패딩)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [12]:
# 평가 메트릭 정의 (F1 스코어)
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels, average='macro')


# 훈련 인자 설정
training_args = TrainingArguments(
            output_dir=self.model_path,
            overwrite_output_dir=True,
            do_train=True,
            do_eval=True,
            do_predict=True,
            logging_strategy='epoch',
            eval_strategy='epoch',
            save_strategy='epoch',
            logging_steps=100,
            # eval_steps=100,
            # save_steps=100,
            save_total_limit=1,
            learning_rate= 2e-05, # 가능
            adam_beta1 = 0.9, # 불가
            adam_beta2 = 0.999, # 불가
            adam_epsilon=1e-08, # 불가
            weight_decay=0.01, # 불가
            lr_scheduler_type='linear', # 불가
            per_device_train_batch_size=32, # 가능
            per_device_eval_batch_size=32, # 32인 건 이유가 있다.
            num_train_epochs=2, # 불가
            # load_best_model_at_end=True,
            metric_for_best_model='eval_f1',
            greater_is_better=True,
            seed=SEED, # 불가? 가급적 건드리지 말기
)


# 모델 초기화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(DEVICE)

# Trainer 설정 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_noisy,
    #eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

# 학습 완료 후 성능 평가
print("\n모델 1 성능 평가:")
eval_results = trainer.evaluate(eval_dataset=valid_dataset)
print(f"F1 Score: {eval_results['eval_f1']:.4f}")

# 성능 저장
model1_f1 = eval_results['eval_f1']

NameError: name 'self' is not defined